In [1]:
!git clone https://github.com/selimsef/dfdc_deepfake_challenge.git

Cloning into 'dfdc_deepfake_challenge'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 90 (delta 9), reused 6 (delta 6), pack-reused 67
Receiving objects: 100% (90/90), 69.16 MiB | 12.77 MiB/s, done.
Resolving deltas: 100% (26/26), done.


In [ ]:
%cd dfdc_deepfake_challenge/
!wget -O weights/final_111_DeepFakeClassifier_tf_efficientnet_b7_ns_0_36 https://github.com/selimsef/dfdc_deepfake_challenge/releases/download/0.0.1/final_111_DeepFakeClassifier_tf_efficientnet_b7_ns_0_36
!wget -O weights/final_555_DeepFakeClassifier_tf_efficientnet_b7_ns_0_19 https://github.com/selimsef/dfdc_deepfake_challenge/releases/download/0.0.1/final_555_DeepFakeClassifier_tf_efficientnet_b7_ns_0_19
!wget -O weights/final_777_DeepFakeClassifier_tf_efficientnet_b7_ns_0_29 https://github.com/selimsef/dfdc_deepfake_challenge/releases/download/0.0.1/final_777_DeepFakeClassifier_tf_efficientnet_b7_ns_0_29
!wget -O weights/final_777_DeepFakeClassifier_tf_efficientnet_b7_ns_0_31 https://github.com/selimsef/dfdc_deepfake_challenge/releases/download/0.0.1/final_777_DeepFakeClassifier_tf_efficientnet_b7_ns_0_31
!wget -O weights/final_888_DeepFakeClassifier_tf_efficientnet_b7_ns_0_37 https://github.com/selimsef/dfdc_deepfake_challenge/releases/download/0.0.1/final_888_DeepFakeClassifier_tf_efficientnet_b7_ns_0_37
!wget -O weights/final_888_DeepFakeClassifier_tf_efficientnet_b7_ns_0_40 https://github.com/selimsef/dfdc_deepfake_challenge/releases/download/0.0.1/final_888_DeepFakeClassifier_tf_efficientnet_b7_ns_0_40
!wget -O weights/final_999_DeepFakeClassifier_tf_efficientnet_b7_ns_0_23 https://github.com/selimsef/dfdc_deepfake_challenge/releases/download/0.0.1/final_999_DeepFakeClassifier_tf_efficientnet_b7_ns_0_23

In [2]:
!pip install timm facenet-pytorch

In [3]:
import argparse
import os
import re
import time

import torch
import pandas as pd
from kernel_utils import VideoReader, FaceExtractor, confident_strategy, predict_on_video
from training.zoo.classifiers import DeepFakeClassifier

In [4]:
models = [
    "final_111_DeepFakeClassifier_tf_efficientnet_b7_ns_0_36",
    "final_555_DeepFakeClassifier_tf_efficientnet_b7_ns_0_19",
    "final_777_DeepFakeClassifier_tf_efficientnet_b7_ns_0_29",
    "final_777_DeepFakeClassifier_tf_efficientnet_b7_ns_0_31",
    "final_888_DeepFakeClassifier_tf_efficientnet_b7_ns_0_37",
    "final_888_DeepFakeClassifier_tf_efficientnet_b7_ns_0_40",
    "final_999_DeepFakeClassifier_tf_efficientnet_b7_ns_0_23"
]
weights_dir = "weights"

In [5]:

models_list = []
model_paths = [os.path.join(weights_dir, model) for model in models]
for path in model_paths:
    model = DeepFakeClassifier(encoder="tf_efficientnet_b7_ns").to("cuda")
    print("loading state dict {}".format(path))
    checkpoint = torch.load(path, map_location="cpu")
    state_dict = checkpoint.get("state_dict", checkpoint)
    model.load_state_dict({re.sub("^module.", "", k): v for k, v in state_dict.items()}, strict=True)
    model.eval()
    del checkpoint
    models_list.append(model.half())

/content/dfdc_deepfake_challenge/training/zoo/classifiers.py:144: UserWarning: Mapping deprecated model name tf_efficientnet_b7_ns to current tf_efficientnet_b7.ns_jft_in1k.
  self.encoder = encoder_params[encoder]["init_op"]()


loading state dict weights/final_111_DeepFakeClassifier_tf_efficientnet_b7_ns_0_36
loading state dict weights/final_555_DeepFakeClassifier_tf_efficientnet_b7_ns_0_19
loading state dict weights/final_777_DeepFakeClassifier_tf_efficientnet_b7_ns_0_29
loading state dict weights/final_777_DeepFakeClassifier_tf_efficientnet_b7_ns_0_31
loading state dict weights/final_888_DeepFakeClassifier_tf_efficientnet_b7_ns_0_37
loading state dict weights/final_888_DeepFakeClassifier_tf_efficientnet_b7_ns_0_40
loading state dict weights/final_999_DeepFakeClassifier_tf_efficientnet_b7_ns_0_23


In [6]:
def predict_test_video(video_path=""):
    video_reader = VideoReader()
    video_read_fn = lambda x: video_reader.read_frames(x, num_frames=32)
    face_extractor = FaceExtractor(video_read_fn)
    input_size = 380
    strategy = confident_strategy
    stime = time.time()

    predictions = predict_on_video(face_extractor=face_extractor, input_size=input_size, models=models_list,
                                       strategy=strategy, video_path=video_path, batch_size=32)
    print("Elapsed:", time.time() - stime)
    return predictions

In [9]:
#0 - real
#1 - fake
#trained on log loss
print(predict_test_video("../input.mp4"))
print(predict_test_video("../input(1).mp4"))
print(predict_test_video("../input(2).mp4"))
print(predict_test_video("../input(3).mp4"))

Elapsed: 3.7595601081848145
0.06152
Elapsed: 7.5848612785339355
0.989
Elapsed: 7.476138591766357
0.1813
Elapsed: 6.9994590282440186
0.9907
